In [10]:
# Load packages
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import os
import scipy as scp
import scipy.stats as scps
from datetime import datetime

# Load my own functions
import dnnregressor_train_eval_keras as dnnk
import make_data_wfpt as mdw

In [2]:
# Load data
data = pd.read_csv(os.getcwd() + '/data_storage/data_11000000_from_simulation_mix_09_12_18_18_20_50.csv')

In [3]:
# Some cleaning of the data
data = data[['v', 'a', 'w', 'rt', 'choice', 'nf_likelihood']]
data = data.loc[data['w'] > 0.1]
data = data.loc[data['w'] < 0.9]
data = data.loc[data['a'] > 0.5]

mini_data = data.loc[1:10000]


train_f, train_l, test_f, test_l = mdw.train_test_split_rt_choice(data = data,
                                                                  write_to_file = False,
                                                                  from_file = False,
                                                                  p_train = 0.8,
                                                                  backend = 'keras')
# Choice probabilities
# train_f, train_l, test_f, test_l = mdw.train_test_from_file_choice_probabilities(n_samples = 2500000,
#                                                             f_signature = '_choice_probabilities_analytic_',
#                                                                                 backend = 'keras')

# rt_choice
# train_f, train_l, test_f, test_l = mdw.train_test_from_file_rt_choice(n_samples = 11000000,
#                                                                       f_signature = '_from_simulation_mix_',
#                                                                       backend = 'keras')

In [4]:
# Make dnnk class (cpm for choice probability model)
cpm = dnnk.dnn_trainer()
cpm.data['train_features'] = train_f
cpm.data['train_labels'] = train_l
cpm.data['test_features'] = test_f
cpm.data['test_labels'] = test_l

In [5]:
# Make all parameters we can specify explicit
# Model parameters
cpm.model_params

{'input_shape': 3,
 'output_shape': 1,
 'output_activation': 'sigmoid',
 'hidden_layers': [20, 20, 20, 20],
 'hidden_activations': ['relu', 'relu', 'relu', 'relu'],
 'l1_activation': [0.0, 0.0, 0.0, 0.0],
 'l2_activation': [0.0, 0.0, 0.0, 0.0],
 'l1_kernel': [0.0, 0.0, 0.0, 0.0],
 'l2_kernel': [0.0, 0.0, 0.0, 0.0],
 'optimizer': 'Nadam',
 'loss': 'mse',
 'metrics': ['mse']}

In [6]:
# Parameters governing training
cpm.train_params

{'callback_funs': ['ReduceLROnPlateau', 'EarlyStopping', 'ModelCheckpoint'],
 'plateau_patience': 10,
 'min_delta': 0.0001,
 'early_stopping_patience': 15,
 'callback_monitor': 'loss',
 'min_learning_rate': 1e-07,
 'red_coef_learning_rate': 0.1,
 'ckpt_period': 10,
 'ckpt_save_best_only': True,
 'ckpt_save_weights_only': True,
 'max_train_epochs': 2000,
 'batch_size': 10000,
 'warm_start': False,
 'checkpoint': 'ckpt',
 'model_cnt': 0}

In [7]:
# Parameters concerning data storage
cpm.data_params

{'data_type': 'choice_probabilities',
 'model_directory': '/home/afengler/git_repos/nn_likelihoods/keras_models',
 'checkpoint': 'ckpt',
 'model_name': 'dnnregressor',
 'data_type_signature': '_choice_probabilities_analytic_',
 'timestamp': '09_15_18_23_59_14',
 'training_data_size': 2500000}

In [8]:
# SPECIFYING META PARAMETERS THAT STAY CONSTANT DURING HYPERPARAMETER OPTIMIZATION

# Model params
cpm.model_params['output_activation'] = 'linear'
cpm.model_params['input_shape'] = 5

# Training params
# Meta
cpm.train_params['early_stopping_patience'] = 5
cpm.train_params['plateau_patience'] = 3
cpm.train_params['min_delta'] = 0.001
cpm.train_params['ckpt_period'] = 1
cpm.train_params['model_cnt'] = 0
cpm.train_params['max_train_epochs'] = 25

# Hyper
#cpm.train_params['l1_kernel']
cpm.model_params['hidden_layers'] = [5, 5, 5, 5]
#cpm.train_params['hidden_activations']
#cpm.train_params['l2_kernel'] = [0.5, 0.5, 0.5, 0.5]
#cpm.train_params['l2_activation'] = [0.5, 0.5, 0.5, 0.5]

# Data params
cpm.data_params['data_type'] = 'wfpt'
cpm.data_params['data_type_signature'] = '_choice_rt_'
cpm.data_params['training_data_size'] = 11000000

# Update timestamp
cpm.data_params['timestamp'] = datetime.now().strftime('%m_%d_%y_%H_%M_%S')

In [ ]:
# Make model
cpm.keras_model_generate(save_model = True)

In [ ]:
# Train model
cpm.run_training(save_history = True, 
                 warm_start = False)

In [9]:
# Hyperparameter training loop:

# Runs: 
num_runs = 20
cnt = 0
max_layers = 5
layer_sizes = [10, 20, 50]
batch_sizes = [1000, 10000, 50000]

# Update model directory to make sure we collect all our models from this hyperparameter optimization run in the same place
cpm.data_params['model_directory'] =  '/home/afengler/git_repos/nn_likelihoods/keras_models/'
cpm.data_params['model_name'] = 'dnnregressor_wftp_hyp_opt'
cpm.train_params['model_cnt'] = 0

while cnt < num_runs:
    cnt += 1
    
    # Sample # layers 
    num_layers = np.random.choice(np.arange(1, max_layers, 1))
    
    # Layer sizes
    layers = []
    activations = []
    regularizers = []
    
    for i in range(0, num_layers, 1):
        layers.append(np.random.choice(layer_sizes))
        activations.append('relu')
        regularizers.append(0.0)
        
    # Batch size
    batch_size = np.random.choice(batch_sizes)
    
    # Update relevant model parameters
    cpm.train_params['batch_size'] = batch_size
    cpm.model_params['hidden_layers'] = layers
    cpm.model_params['hidden_activations'] = activations
    cpm.model_params['l1_activation'] = regularizers
    cpm.model_params['l2_activation'] = regularizers
    cpm.model_params['l1_kernel'] = regularizers
    cpm.model_params['l2_kernel'] = regularizers
    
    # Increment model count
    cpm.data_params['model_cnt'] += 1
    
    # Make new timestamp
    #cpm.data_params['timestamp'] = datetime.now().strftime('%m_%d_%y_%H_%M_%S')
    
    # Make model
    cpm.keras_model_generate(save_model = True)
    
    # Train model
    histories.append(cpm.run_training(save_history = True, 
                                    warm_start = False))
    
    histories[-1]['model_cnt'] = cpm.train_params['model_cnt']
    histories[-1]['num_layers'] = num_layers
    histories[-1]['size_layers'] = str(layers)
    histories[-1]['activations'] = str(activations) 
    histories[-1]['batch_size'] = batch_size
    
    print(cnt)
    
histories = pd.concat(histories)
histories['optimizer'] = cpm.model_params['optimizer']
histories['timestamp'] = datetime.now().strftime('%m_%d_%y_%H_%M_%S')
histories.to_csv(cpm.data_params['model_directory'] + cpm.data_params['model_name'] + '_choice_rt_' +\
                 cpm.data_params['timestamp'] + '/hyp_opt_histories.csv')

Train on 6072894 samples, validate on 1515113 samples
Epoch 1/100
6072894/6072894 [==============================] - 186s 31us/step - loss: 2.1434 - mean_squared_error: 2.1434 - val_loss: 1.0423 - val_mean_squared_error: 1.0423

Epoch 00001: val_loss improved from inf to 1.04228, saving model to /home/afengler/git_repos/nn_likelihoods/keras_models//dnnregressor_wftp_hyp_opt_choice_rt_09_15_18_23_59_39/ckpt-0-01
Epoch 2/100
6072894/6072894 [==============================] - 186s 31us/step - loss: 1.1171 - mean_squared_error: 1.1171 - val_loss: 2.1657 - val_mean_squared_error: 2.1657

Epoch 00002: val_loss did not improve from 1.04228
Epoch 3/100
6072894/6072894 [==============================] - 187s 31us/step - loss: 0.8194 - mean_squared_error: 0.8194 - val_loss: 0.4458 - val_mean_squared_error: 0.4458

Epoch 00003: val_loss improved from 1.04228 to 0.44582, saving model to /home/afengler/git_repos/nn_likelihoods/keras_models//dnnregressor_wftp_hyp_opt_choice_rt_09_15_18_23_59_39/ckpt


Epoch 00028: val_loss did not improve from 0.08674
Epoch 29/100
6072894/6072894 [==============================] - 185s 31us/step - loss: 0.0850 - mean_squared_error: 0.0850 - val_loss: 0.0832 - val_mean_squared_error: 0.0832

Epoch 00029: val_loss improved from 0.08674 to 0.08320, saving model to /home/afengler/git_repos/nn_likelihoods/keras_models//dnnregressor_wftp_hyp_opt_choice_rt_09_15_18_23_59_39/ckpt-0-29
Epoch 30/100
6072894/6072894 [==============================] - 186s 31us/step - loss: 0.0842 - mean_squared_error: 0.0842 - val_loss: 0.0983 - val_mean_squared_error: 0.0983

Epoch 00030: val_loss did not improve from 0.08320
Epoch 31/100
6072894/6072894 [==============================] - 186s 31us/step - loss: 0.0834 - mean_squared_error: 0.0834 - val_loss: 0.0836 - val_mean_squared_error: 0.0836

Epoch 00031: val_loss did not improve from 0.08320
Epoch 32/100
6072894/6072894 [==============================] - 185s 30us/step - loss: 0.0821 - mean_squared_error: 0.0821 - val

6072894/6072894 [==============================] - 186s 31us/step - loss: 0.0552 - mean_squared_error: 0.0552 - val_loss: 0.0520 - val_mean_squared_error: 0.0520

Epoch 00055: val_loss did not improve from 0.05108
Epoch 56/100
6072894/6072894 [==============================] - 185s 30us/step - loss: 0.0543 - mean_squared_error: 0.0543 - val_loss: 0.0515 - val_mean_squared_error: 0.0515

Epoch 00056: val_loss did not improve from 0.05108
Epoch 57/100
6072894/6072894 [==============================] - 186s 31us/step - loss: 0.0532 - mean_squared_error: 0.0532 - val_loss: 0.0538 - val_mean_squared_error: 0.0538

Epoch 00057: val_loss did not improve from 0.05108
Epoch 58/100
6072894/6072894 [==============================] - 185s 30us/step - loss: 0.0525 - mean_squared_error: 0.0525 - val_loss: 0.0496 - val_mean_squared_error: 0.0496

Epoch 00058: val_loss improved from 0.05108 to 0.04956, saving model to /home/afengler/git_repos/nn_likelihoods/keras_models//dnnregressor_wftp_hyp_opt_choi

6072894/6072894 [==============================] - 186s 31us/step - loss: 0.0378 - mean_squared_error: 0.0378 - val_loss: 0.0380 - val_mean_squared_error: 0.0380

Epoch 00084: val_loss did not improve from 0.03484
Epoch 85/100
6072894/6072894 [==============================] - 185s 31us/step - loss: 0.0373 - mean_squared_error: 0.0373 - val_loss: 0.0365 - val_mean_squared_error: 0.0365

Epoch 00085: val_loss did not improve from 0.03484
Epoch 86/100
6072894/6072894 [==============================] - 185s 30us/step - loss: 0.0370 - mean_squared_error: 0.0370 - val_loss: 0.0356 - val_mean_squared_error: 0.0356

Epoch 00086: val_loss did not improve from 0.03484
Epoch 87/100
6072894/6072894 [==============================] - 186s 31us/step - loss: 0.0364 - mean_squared_error: 0.0364 - val_loss: 0.0361 - val_mean_squared_error: 0.0361

Epoch 00087: val_loss did not improve from 0.03484
Epoch 88/100
6072894/6072894 [==============================] - 186s 31us/step - loss: 0.0365 - mean_squa

6072894/6072894 [==============================] - 173s 28us/step - loss: 0.3213 - mean_squared_error: 0.3213 - val_loss: 0.1995 - val_mean_squared_error: 0.1995

Epoch 00013: val_loss did not improve from 0.19248
Epoch 14/100
6072894/6072894 [==============================] - 172s 28us/step - loss: 0.3022 - mean_squared_error: 0.3022 - val_loss: 0.5720 - val_mean_squared_error: 0.5720

Epoch 00014: val_loss did not improve from 0.19248
Epoch 15/100
6072894/6072894 [==============================] - 173s 28us/step - loss: 0.2890 - mean_squared_error: 0.2890 - val_loss: 0.4032 - val_mean_squared_error: 0.4032

Epoch 00015: val_loss did not improve from 0.19248
Epoch 16/100
6072894/6072894 [==============================] - 173s 28us/step - loss: 0.2853 - mean_squared_error: 0.2853 - val_loss: 0.4080 - val_mean_squared_error: 0.4080

Epoch 00016: val_loss did not improve from 0.19248
Epoch 17/100
6072894/6072894 [==============================] - 173s 29us/step - loss: 0.2837 - mean_squa

...

6072894/6072894 [==============================] - 163s 27us/step - loss: 5.7855 - mean_squared_error: 5.7855 - val_loss: 5.6034 - val_mean_squared_error: 5.6034

Epoch 00026: val_loss improved from 5.73087 to 5.60339, saving model to /home/afengler/git_repos/nn_likelihoods/keras_models//dnnregressor_wftp_hyp_opt_choice_rt_09_15_18_23_59_39/ckpt-18-26
Epoch 27/100
6072894/6072894 [==============================] - 159s 26us/step - loss: 5.7600 - mean_squared_error: 5.7600 - val_loss: 5.6367 - val_mean_squared_error: 5.6367

Epoch 00027: val_loss did not improve from 5.60339
Epoch 28/100
6072894/6072894 [==============================] - 163s 27us/step - loss: 5.7434 - mean_squared_error: 5.7434 - val_loss: 5.6441 - val_mean_squared_error: 5.6441

Epoch 00028: val_loss did not improve from 5.60339
Epoch 29/100
6072894/6072894 [==============================] - 159s 26us/step - loss: 5.7284 - mean_squared_error: 5.7284 - val_loss: 5.5741 - val_mean_squared_error: 5.5741

Epoch 00029: val

6072894/6072894 [==============================] - 155s 26us/step - loss: 5.4712 - mean_squared_error: 5.4712 - val_loss: 5.3054 - val_mean_squared_error: 5.3054

Epoch 00059: val_loss improved from 5.39015 to 5.30543, saving model to /home/afengler/git_repos/nn_likelihoods/keras_models//dnnregressor_wftp_hyp_opt_choice_rt_09_15_18_23_59_39/ckpt-18-59
Epoch 60/100
6072894/6072894 [==============================] - 156s 26us/step - loss: 5.4660 - mean_squared_error: 5.4660 - val_loss: 5.3574 - val_mean_squared_error: 5.3574

Epoch 00060: val_loss did not improve from 5.30543
Epoch 61/100
6072894/6072894 [==============================] - 155s 26us/step - loss: 5.4633 - mean_squared_error: 5.4633 - val_loss: 5.4156 - val_mean_squared_error: 5.4156

Epoch 00061: val_loss did not improve from 5.30543
Epoch 62/100
6072894/6072894 [==============================] - 155s 26us/step - loss: 5.4585 - mean_squared_error: 5.4585 - val_loss: 5.4183 - val_mean_squared_error: 5.4183

Epoch 00062: val

6072894/6072894 [==============================] - 155s 26us/step - loss: 5.2681 - mean_squared_error: 5.2681 - val_loss: 5.3132 - val_mean_squared_error: 5.3132

Epoch 00092: val_loss did not improve from 5.10607
Epoch 93/100
6072894/6072894 [==============================] - 155s 26us/step - loss: 5.2621 - mean_squared_error: 5.2621 - val_loss: 5.3222 - val_mean_squared_error: 5.3222

Epoch 00093: val_loss did not improve from 5.10607
Epoch 94/100
6072894/6072894 [==============================] - 156s 26us/step - loss: 5.2582 - mean_squared_error: 5.2582 - val_loss: 5.2650 - val_mean_squared_error: 5.2650

Epoch 00094: val_loss did not improve from 5.10607
Epoch 95/100
6072894/6072894 [==============================] - 155s 25us/step - loss: 5.2538 - mean_squared_error: 5.2538 - val_loss: 5.2826 - val_mean_squared_error: 5.2826

Epoch 00095: val_loss did not improve from 5.10607
Epoch 96/100
6072894/6072894 [==============================] - 155s 26us/step - loss: 5.2507 - mean_squa

6072894/6072894 [==============================] - 176s 29us/step - loss: 0.2091 - mean_squared_error: 0.2091 - val_loss: 0.1176 - val_mean_squared_error: 0.1176

Epoch 00022: val_loss improved from 0.12419 to 0.11757, saving model to /home/afengler/git_repos/nn_likelihoods/keras_models//dnnregressor_wftp_hyp_opt_choice_rt_09_15_18_23_59_39/ckpt-19-22
Epoch 23/100
6072894/6072894 [==============================] - 178s 29us/step - loss: 0.2068 - mean_squared_error: 0.2068 - val_loss: 0.1413 - val_mean_squared_error: 0.1413

Epoch 00023: val_loss did not improve from 0.11757
Epoch 24/100
6072894/6072894 [==============================] - 177s 29us/step - loss: 0.2040 - mean_squared_error: 0.2040 - val_loss: 0.1244 - val_mean_squared_error: 0.1244

Epoch 00024: val_loss did not improve from 0.11757
Epoch 25/100
6072894/6072894 [==============================] - 178s 29us/step - loss: 0.2003 - mean_squared_error: 0.2003 - val_loss: 0.1363 - val_mean_squared_error: 0.1363

Epoch 00025: val

6072894/6072894 [==============================] - 176s 29us/step - loss: 0.1460 - mean_squared_error: 0.1460 - val_loss: 0.0966 - val_mean_squared_error: 0.0966

Epoch 00055: val_loss did not improve from 0.09365
Epoch 56/100
6072894/6072894 [==============================] - 178s 29us/step - loss: 0.1470 - mean_squared_error: 0.1470 - val_loss: 0.1005 - val_mean_squared_error: 0.1005

Epoch 00056: val_loss did not improve from 0.09365
Epoch 57/100
6072894/6072894 [==============================] - 179s 29us/step - loss: 0.1481 - mean_squared_error: 0.1481 - val_loss: 0.2066 - val_mean_squared_error: 0.2066

Epoch 00057: val_loss did not improve from 0.09365
Epoch 58/100
6072894/6072894 [==============================] - 177s 29us/step - loss: 0.1463 - mean_squared_error: 0.1463 - val_loss: 0.1013 - val_mean_squared_error: 0.1013

Epoch 00058: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.

Epoch 00058: val_loss did not improve from 0.09365
Epoch 00058: early stop

In [19]:
cpm.train_params['min_delta'] = 0.001